In [8]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('wordnet')
nltk.download('stopwords')

# Load dictionary and test data
dictionary_path = 'Dictionary 10.xlsx'
test_data_path = 'TEST DATA.csv'

# Read dictionary from excel file
dictionary_df = pd.read_excel(dictionary_path)

# Read test data from csv file
test_data_df = pd.read_csv(test_data_path)

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Preprocess text: lowercase, remove stopwords, and lemmatize
def preprocess(text):
    return ' '.join(
        lemmatizer.lemmatize(word)
        for word in text.lower().split()
        if word not in stop_words
    )

# Preprocess dictionary words
dictionary_df = dictionary_df.applymap(lambda x: preprocess(x) if isinstance(x, str) else x)

# Preprocess test data sentences
test_data_df['Text'] = test_data_df['Text'].astype(str).apply(preprocess)

# Initialize output dataframe with the necessary columns
output_df = test_data_df[['Text-ID', 'Sentence-ID']].copy()

# Flatten the dictionary to create a unique vocabulary list
vocabulary = dictionary_df.values.flatten()
vocabulary = list(set([word for word in vocabulary if pd.notna(word)]))

# Initialize TfidfVectorizer with the unique vocabulary
vectorizer = TfidfVectorizer(vocabulary=vocabulary, lowercase=False)

# Fit and transform the text data
tfidf_matrix = vectorizer.fit_transform(test_data_df['Text'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Merge the TF-IDF scores with the output DataFrame
output_df = pd.concat([output_df, tfidf_df], axis=1)

# Initialize dictionary count columns with zeros
behavior_columns = ["Security", "Conformity", "Tradition", "Benevolence", "Universalism",
                    "Self-Direction", "Stimulation", "Hedonism", "Achievement", "Power"]

for column in behavior_columns:
    output_df[column] = 0

# Function to count matches
def count_matches(sentence, words_list):
    sentence_words = set(sentence.lower().split())
    return sum(word in sentence_words for word in words_list)

# Update output dataframe with counts
for idx, row in test_data_df.iterrows():
    sentence = row['Text']  # Assuming the sentence is in the 'Text' column
    for column in behavior_columns:
        words_list = dictionary_df[column].dropna().str.lower().tolist()
        output_df.at[idx, column] = count_matches(sentence, words_list)

# Select only the required columns for the output
output_df = output_df[['Text-ID', 'Sentence-ID'] + behavior_columns]

# Save the output dataframe to a new CSV file
output_file_path = 'Filtered_Output.csv'
output_df.to_csv(output_file_path, index=False)

output_df.head()


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rites\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rites\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\rites\AppData\Local\Temp\ipykernel_14856\4292542051.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dictionary_df = dictionary_df.applymap(lambda x: preprocess(x) if isinstance(x, str) else x)


,Text-ID,Sentence-ID,Security,Conformity,Tradition,Benevolence,Universalism,Self-Direction,Stimulation,Hedonism,Achievement,Power
0,BG_002,1,0,0,0,0,0,0,0,0,0,0
1,BG_002,2,0,0,0,0,0,0,0,0,0,0
2,BG_002,3,0,1,0,0,0,0,0,0,0,2
3,BG_002,4,0,0,0,0,0,0,0,0,0,0
4,BG_002,5,0,0,0,0,2,1,0,1,0,0


In [5]:
import pandas as pd

# Load the data from the Excel file
file_path = 'Filtered_Output.csv'  # Replace with the path to your file
df = pd.read_csv(file_path)


# List of behavior columns to rank
behavior_columns = ['Security', 'Conformity', 'Tradition', 'Benevolence', 
                    'Universalism', 'Self-Direction', 'Stimulation', 
                    'Hedonism', 'Achievement', 'Power']

# Function to rank a row's values and assign rank to each behavior
def rank_row(row):
    # Replace 0s with pd.NA to exclude them from ranking
    row_no_zero = row.replace(0, pd.NA)
    # Rank the values, with the largest number being ranked 1, handle ties
    ranks = row_no_zero.rank(method='min', ascending=False)
    
    # Prepare a dictionary for storing rank information
    rank_dict = {}
    
    # Dictionary to store behaviors corresponding to each rank (Rank_1, Rank_2, ...)
    rank_behavior_dict = {f'Rank_{i}': '' for i in range(1, 11)}
    
    # Loop over each behavior and assign ranks
    for behavior in behavior_columns:
        rank = ranks.get(behavior, None)
        if pd.notna(rank):
            # If the rank already has values, append this behavior
            if rank_behavior_dict[f'Rank_{int(rank)}']:
                rank_behavior_dict[f'Rank_{int(rank)}'] += ', ' + behavior
            else:
                rank_behavior_dict[f'Rank_{int(rank)}'] = behavior

    # Add rank information for each behavior column
    for col in behavior_columns:
        rank_dict[f'{col}_Rank'] = ranks.get(col, ' ') if pd.notna(ranks.get(col)) else ' '

    # Combine both dictionaries (rank behavior and rank columns)
    return pd.Series({**rank_dict, **rank_behavior_dict})

# Apply ranking function to each row for the behavior columns
ranked_df = df[behavior_columns].apply(rank_row, axis=1)

# Add columns for behavior rank and Rank_1 to Rank_10 values to the original dataframe
df = pd.concat([df, ranked_df], axis=1)

# Calculate the sum of the original behavior scores for each row
df['Sum_of_Scores'] = df[behavior_columns].sum(axis=1)

# Save the updated dataframe back to a new Excel file
output_file = 'final_ranked_output_with_behaviors_TFIDF.xlsx'  # Specify the output file name
df.to_excel(output_file, index=False)

print(f"Ranking and additional 21 columns added successfully and saved to {output_file}")

Ranking and additional 21 columns added successfully and saved to final_ranked_output_with_behaviors_TFIDF.xlsx


In [7]:
import pandas as pd

# Load the previously generated file with ranks
final_ranks_file = 'final_ranked_output_with_behaviors_TFIDF.xlsx'
df_final = pd.read_excel(final_ranks_file)

# Load the expert file (assume same structure as original data)
expert_file = 'TEST SCORE UPD.csv'  # Update with actual expert file path
df_expert = pd.read_csv(expert_file)

# List of behavior columns
behavior_columns = ['Security', 'Conformity', 'Tradition', 'Benevolence', 
                    'Universalism', 'Self-Direction', 'Stimulation', 
                    'Hedonism', 'Achievement', 'Power']

# Merge the two dataframes on Text-ID and Sentence-ID
df_merged = pd.merge(df_expert, df_final, on=['Text-ID', 'Sentence-ID'], suffixes=('_expert', '_final'))

# Function to find human value from expert data and its corresponding rank
def find_expert_value_and_rank(row):
    # Find the human value based on the expert data (any column where value is greater than 0)
    human_value = ', '.join([col for col in behavior_columns if row[f'{col}_expert'] > 0])
    
    # If no human value found, return blank and None for the rank
    if not human_value:
        return pd.Series({'Expert_Human_Value': '', 'Expert_Value_Rank': ''})
    
    # Find the rank corresponding to the human value(s) from the final ranks
    rank_list = []
    for value in human_value.split(', '):
        rank = row[f'{value}_Rank']
        rank_list.append(str(rank))
    
    # Join all ranks if multiple human values
    rank_value = ', '.join(rank_list) if rank_list else ''
    
    return pd.Series({'Expert_Human_Value': human_value, 'Expert_Value_Rank': rank_value})

# Apply the function to each row to get the expert human value and rank
df_merged[['Expert_Human_Value', 'Expert_Value_Rank']] = df_merged.apply(find_expert_value_and_rank, axis=1)

# Save the updated dataframe with the expert analysis columns
output_file = 'final_output_with_expert_analysis_TFIDF.xlsx'
df_merged.to_excel(output_file, index=False)

print(f"Expert data successfully merged and saved to {output_file}")


Expert data successfully merged and saved to final_output_with_expert_analysis_TFIDF.xlsx


In [12]:
import pandas as pd

# Load the previously generated output file
output_file_path = 'Filtered_Output.csv'  # Update this path if necessary
output_df = pd.read_csv(output_file_path)

# Define the columns to process
columns_to_process = ["Self-Direction", "Stimulation", "Hedonism", "Achievement", "Power", "Security",
                      "Tradition", "Conformity", "Benevolence", "Universalism"]

# Iterate over each row
for idx, row in output_df.iterrows():
    # Check if all values are 0
    if (row[columns_to_process] == 0).all():
        continue  # Skip updating this row if all values are 0
    
    # Find the maximum value for the row
    max_value = row[columns_to_process].max()
    
    # Update columns: set to 1 if equal to max value, else 0
    for column in columns_to_process:
        output_df.at[idx, column] = 1 if row[column] == max_value else 0

# Save the updated output dataframe to a new CSV file
updated_output_file_path = 'TF_IDF_Output1.csv'  # Update this path if necessary
output_df.to_csv(updated_output_file_path, index=False)

output_df.head()


,Text-ID,Sentence-ID,Security,Conformity,Tradition,Benevolence,Universalism,Self-Direction,Stimulation,Hedonism,Achievement,Power
0,BG_002,1,0,0,0,0,0,0,0,0,0,0
1,BG_002,2,0,0,0,0,0,0,0,0,0,0
2,BG_002,3,0,0,0,0,0,0,0,0,0,1
3,BG_002,4,0,0,0,0,0,0,0,0,0,0
4,BG_002,5,0,0,0,0,1,0,0,0,0,0


In [14]:
import pandas as pd

# Load the test score and updated output files
test_score_path = 'TEST SCORE UPD.csv'  # Update this path if necessary
updated_output_path = 'TF_IDF_Output1.csv'  # Update this path if necessary

test_score_df = pd.read_csv(test_score_path)
updated_output_df = pd.read_csv(updated_output_path)

# Print the column names to check for discrepancies
print("Test Score Columns:")
print(test_score_df.columns)

print("\nUpdated Output Columns:")
print(updated_output_df.columns)

# Define the columns to process (update these if necessary based on the above output)
columns_to_process = ["Security", "Conformity", "Tradition", "Benevolence", "Universalism", 
                      "Self-Direction", "Stimulation", "Hedonism", "Achievement", "Power"]

# Merge dataframes based on Text-ID and Sentence-ID
merged_df = test_score_df.merge(updated_output_df, on=['Text-ID', 'Sentence-ID'], suffixes=('_test', '_updated'))

# Function to find headers with value 1
def find_headers_with_one(row, suffix):
    return ','.join([col for col in columns_to_process if row[f"{col}{suffix}"] == 1])

# Add new columns for headers with value 1
merged_df['Headers_1_test'] = merged_df.apply(lambda row: find_headers_with_one(row, '_test'), axis=1)
merged_df['Headers_1_updated'] = merged_df.apply(lambda row: find_headers_with_one(row, '_updated'), axis=1)

# Reorder columns
final_columns = ['Text-ID', 'Sentence-ID'] + \
                [f"{col}_test" for col in columns_to_process] + \
                [f"{col}_updated" for col in columns_to_process] + \
                ['Headers_1_test', 'Headers_1_updated']

final_df = merged_df[final_columns]

# Save to a new Excel file
output_excel_path = 'TF_IDF_Output1_combined.xlsx'  # Update this path if necessary
final_df.to_excel(output_excel_path, index=False)

final_df.head()


Test Score Columns:
Index(['Text-ID', 'Sentence-ID', 'Security', 'Conformity', 'Tradition',
       'Benevolence', 'Universalism', 'Self-Direction', 'Stimulation',
       'Hedonism', 'Achievement', 'Power'],
      dtype='object')

Updated Output Columns:
Index(['Text-ID', 'Sentence-ID', 'Security', 'Conformity', 'Tradition',
       'Benevolence', 'Universalism', 'Self-Direction', 'Stimulation',
       'Hedonism', 'Achievement', 'Power'],
      dtype='object')


,Text-ID,Sentence-ID,Security_test,Conformity_test,Tradition_test,Benevolence_test,Universalism_test,Self-Direction_test,Stimulation_test,Hedonism_test,...,Tradition_updated,Benevolence_updated,Universalism_updated,Self-Direction_updated,Stimulation_updated,Hedonism_updated,Achievement_updated,Power_updated,Headers_1_test,Headers_1_updated
0,BG_002,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Conformity,
1,BG_002,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Conformity,
2,BG_002,3,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,"Conformity,Hedonism",Power
3,BG_002,4,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Conformity,
4,BG_002,5,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,Conformity,Universalism


In [16]:
import pandas as pd

# Load the combined output file
combined_output_path = 'TF_IDF_Output1_combined.xlsx'
combined_df = pd.read_excel(combined_output_path)

# Function to calculate the match percentage
def calculate_match_percentage(test_headers, updated_headers):
    if pd.isna(test_headers) and pd.isna(updated_headers):
        return 1.0, 1.0
    
    test_words = set(str(test_headers).split(',')) if pd.notna(test_headers) else set()
    updated_words = set(str(updated_headers).split(',')) if pd.notna(updated_headers) else set()
    
    matches = test_words.intersection(updated_words)
    num_matches = len(matches)
    
    test_percentage = num_matches / len(test_words) if test_words else 0.0
    updated_percentage = num_matches / len(updated_words) if updated_words else 0.0
    
    return test_percentage, updated_percentage

# Apply the function to each row and create new columns
combined_df[['Test_Match_Percentage', 'Updated_Match_Percentage']] = combined_df.apply(
    lambda row: calculate_match_percentage(row['Headers_1_test'], row['Headers_1_updated']), axis=1, result_type='expand'
)

# Calculate the total score for each column
total_rows = len(combined_df)
test_total_score = combined_df['Test_Match_Percentage'].sum() / total_rows
updated_total_score = combined_df['Updated_Match_Percentage'].sum() / total_rows

# Add the total score to the dataframe
combined_df.loc['Total'] = combined_df.sum(numeric_only=True)
combined_df.at['Total', 'Test_Match_Percentage'] = test_total_score
combined_df.at['Total', 'Updated_Match_Percentage'] = updated_total_score

# Save the updated dataframe to a new Excel file
updated_combined_output_path = 'TF_IDF_Output1_combined_final.xlsx'
combined_df.to_excel(updated_combined_output_path, index=False)

combined_df.head(), test_total_score, updated_total_score


(  Text-ID  Sentence-ID  Security_test  Conformity_test  Tradition_test  \
 0  BG_002          1.0            0.0              1.0             0.0   
 1  BG_002          2.0            0.0              1.0             0.0   
 2  BG_002          3.0            0.0              1.0             0.0   
 3  BG_002          4.0            0.0              1.0             0.0   
 4  BG_002          5.0            0.0              1.0             0.0   
 
    Benevolence_test  Universalism_test  Self-Direction_test  Stimulation_test  \
 0               0.0                0.0                  0.0               0.0   
 1               0.0                0.0                  0.0               0.0   
 2               0.0                0.0                  0.0               0.0   
 3               0.0                0.0                  0.0               0.0   
 4               0.0                0.0                  0.0               0.0   
 
    Hedonism_test  ...  Universalism_updated  Self-Dir